CREAMOS LAS MÉTRICA PARA EL ANÁLISIS DE VENTAS.

De las tablas ventas podemos determinar:
-los valores totales de cada venta
-la fecha de la primera y última compra
-Compras por cliente.
-Actividad por día, mes y año.

In [1]:
#importamos librerias
import pandas as pd    # Librería para tablas y tratamiento de datos.
#Para una visualización completa de todas las columnas:
pd.set_option('display.max_columns', None)
#Accedemos a las funciones en soporte_dts de la carpeta src
from src import soporte_dts as sdts
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [69]:
df_ventas_cliente_prod=sdts.leer_datos('clientes_productos_ventas_limpio.csv')

In [70]:
df_ventas_cliente_prod.columns

Index(['id_venta', 'id_producto', 'cantidad', 'fecha', 'id_cliente', 'nombre',
       'apellido', 'ciudad', 'precio', 'categoria', 'descripcion', 'pais'],
      dtype='object')

In [53]:
#Añadimos columna para determinar el dia de la semana que se realizan las compras:
df_ventas_cliente_prod['fecha']=pd.to_datetime(df_ventas_cliente_prod['fecha'])

In [21]:
df_ventas_cliente_prod['dia_semana_venta']=df_ventas_cliente_prod['fecha'].dt.dayofweek

In [78]:
#añadimos una nueva columna donde calculamos el valor de cada venta:
df_ventas_cliente_prod['valor_total_venta']=df_ventas_cliente_prod['cantidad']*df_ventas_cliente_prod['precio']
df_ventas_cliente_prod.sample(2)

,id_venta,id_producto,cantidad,fecha,id_cliente,nombre,apellido,ciudad,precio,categoria,descripcion,pais,valor_total_venta
71829,V000252,P0421,6,2021-10-10,C0251,María,Pérez,Oporto,10.38,Books,8YdTEz9VSL3Jvp9BCthxx0FejhFjk3xSOCVhtMhTEiuixH...,Portugal,62.28
66998,V003957,P0071,7,2022-09-17,C0033,María,Hernández,Bilbao,64.03,Books,ne6G8PzuusMbupVEpqRvahDXhiXj555qA3lmXpT97Fuxai...,España,448.21


In [79]:
#Procedemos a guardar el df _limpio:
df_ventas_cliente_prod.to_csv('clientes_productos_ventas_limpio.csv',index=False)

In [56]:
# agruparemos la columna clientes y determinaremos lafecha asociada minima y máxima, el numero de ventas registras, y el 
#Creamos un Dataframe con las métricas que podemos extraer de los datos:
#aplicamos diferentes funcioones de agregación por columna:
metricas_clientes=df_ventas_cliente_prod.groupby('id_cliente', as_index=False).agg({'fecha':['min','max'],'id_venta':'nunique','valor_total_venta':'sum'})
metricas_clientes.columns=['id_cliente','primera_compra','ultima_compra','numero_compras','valor_total']


In [7]:
metricas_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_cliente      210 non-null    object 
 1   primera_compra  210 non-null    object 
 2   ultima_compra   210 non-null    object 
 3   numero_compras  210 non-null    int64  
 4   valor_total     210 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 8.3+ KB


In [57]:
#Obtenemos informacion adicional a partir de las fechas: para ello debemos modificar el tiopo de dat object a datatime:


metricas_clientes['primera_compra']=pd.to_datetime(metricas_clientes['primera_compra'])
metricas_clientes['ultima_compra']=pd.to_datetime(metricas_clientes['ultima_compra'])

In [58]:
#Creamos columnas con año dia y fecha de cada transacción:
metricas_clientes['dia_primera_compra']=metricas_clientes['primera_compra'].dt.day
metricas_clientes['mes_primera_compra']=metricas_clientes['primera_compra'].dt.month
metricas_clientes['año_primera_compra']=metricas_clientes['primera_compra'].dt.year

metricas_clientes['dia_ultima_compra']=metricas_clientes['ultima_compra'].dt.day
metricas_clientes['mes_ultima_compra']=metricas_clientes['ultima_compra'].dt.month
metricas_clientes['año_ultima_compra']=metricas_clientes['ultima_compra'].dt.year

In [59]:
metricas_clientes.sample(5)


,id_cliente,primera_compra,ultima_compra,numero_compras,valor_total,dia_primera_compra,mes_primera_compra,año_primera_compra,dia_ultima_compra,mes_ultima_compra,año_ultima_compra
23,C0037,2020-02-12,2024-05-02,40,93737.76,12,2,2020,2,5,2024
188,C0264,2020-01-29,2024-05-27,41,119542.12,29,1,2020,27,5,2024
28,C0043,2020-01-06,2024-05-16,42,115738.11,6,1,2020,16,5,2024
137,C0199,2020-01-10,2023-11-14,21,57331.39,10,1,2020,14,11,2023
107,C0156,2020-01-16,2024-05-14,41,109147.35,16,1,2020,14,5,2024


In [60]:
#Calculamos la antiguedad en días con respecto a la fecha actual
metricas_clientes['antiguedad_dias']=(datetime.today()-metricas_clientes['primera_compra']).dt.days



In [64]:
#Calculamos el tiempo de actividad y la frecuencia( nºcompras/tiempo de actividad)
metricas_clientes['actividad_dias']=(metricas_clientes['ultima_compra']-metricas_clientes['primera_compra']).dt.days
metricas_clientes['actividad_meses']=metricas_clientes['actividad_dias'].apply(lambda x: round(x/30),1)
metricas_clientes['actividad_años']=metricas_clientes['actividad_meses'].apply(lambda x: round(x/12),1)
metricas_clientes['compras_por_mes']=metricas_clientes['numero_compras']/metricas_clientes['actividad_meses']
metricas_clientes['compras_por_año']=metricas_clientes['numero_compras']/metricas_clientes['actividad_años']

C:\Users\marg_\AppData\Local\Temp\ipykernel_23872\2051430706.py:3: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  metricas_clientes['actividad_meses']=metricas_clientes['actividad_dias'].apply(lambda x: round(x/30),1)
C:\Users\marg_\AppData\Local\Temp\ipykernel_23872\2051430706.py:4: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  metricas_clientes['actividad_años']=metricas_clientes['actividad_meses'].apply(lambda x: round(x/12),1)


In [65]:
metricas_clientes.sample(5)


,id_cliente,primera_compra,ultima_compra,numero_compras,valor_total,dia_primera_compra,mes_primera_compra,año_primera_compra,dia_ultima_compra,mes_ultima_compra,año_ultima_compra,antiguedad_dias,actividad_dias,actividad_meses,actividad_años,compras_por_mes,compras_por_año
204,C0291,2020-02-06,2024-06-21,40,111089.78,6,2,2020,21,6,2024,1804,1597,53,4,0.754717,10.00
179,C0253,2020-01-06,2024-06-03,44,113605.04,6,1,2020,3,6,2024,1835,1610,54,4,0.814815,11.00
190,C0267,2020-01-29,2024-05-10,41,112061.49,29,1,2020,10,5,2024,1812,1563,52,4,0.788462,10.25
32,C0047,2020-02-01,2024-06-09,35,91151.23,1,2,2020,9,6,2024,1809,1590,53,4,0.660377,8.75
54,C0075,2020-04-29,2024-04-15,42,109068.91,29,4,2020,15,4,2024,1721,1447,48,4,0.875000,10.50


In [81]:
metricas_clientes.to_csv('csv_limpios_finales\metricas_cliente.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\marg_\AppData\Local\Temp\ipykernel_23872\1741905565.py:1: SyntaxWarning: invalid escape sequence '\m'
  metricas_clientes.to_csv('csv_limpios_finales\metricas_cliente.csv')


Separamos las columnas correspondientes a información clientes para tenerla en separado:

In [80]:
df_ventas_cliente_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79920 entries, 0 to 79919
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_venta           79920 non-null  object 
 1   id_producto        79920 non-null  object 
 2   cantidad           79920 non-null  int64  
 3   fecha              79920 non-null  object 
 4   id_cliente         79920 non-null  object 
 5   nombre             79920 non-null  object 
 6   apellido           79920 non-null  object 
 7   ciudad             79920 non-null  object 
 8   precio             79920 non-null  float64
 9   categoria          79920 non-null  object 
 10  descripcion        79920 non-null  object 
 11  pais               79920 non-null  object 
 12  valor_total_venta  79920 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 7.9+ MB


In [75]:
df_clientes_informacion=df_ventas_cliente_prod[['id_cliente','nombre','apellido','ciudad','pais']]
df_clientes_informacion.to_csv('csv_limpios_finales\clientes_información.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79920 entries, 0 to 79919
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  79920 non-null  object
 1   nombre      79920 non-null  object
 2   apellido    79920 non-null  object
 3   ciudad      79920 non-null  object
 4   pais        79920 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\marg_\AppData\Local\Temp\ipykernel_23872\570681968.py:3: SyntaxWarning: invalid escape sequence '\c'
  df_clientes_informacion.to_csv('csv_limpios_finales\clientes_información.csv')


In [83]:
df_productos_informacion=df_ventas_cliente_prod[['id_producto','precio','categoria','descripcion']]
df_ventas_informacion=df_ventas_cliente_prod[['id_venta','id_producto','id_cliente','cantidad','fecha','valor_total_venta']]
df_productos_informacion.to_csv('csv_limpios_finales\productos_informacion.csv')
df_ventas_informacion.to_csv('csv_limpios_finales\ ventas_informacion.csv')

<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\ '
C:\Users\marg_\AppData\Local\Temp\ipykernel_23872\2306241212.py:3: SyntaxWarning: invalid escape sequence '\p'
  df_productos_informacion.to_csv('csv_limpios_finales\productos_informacion.csv')
C:\Users\marg_\AppData\Local\Temp\ipykernel_23872\2306241212.py:4: SyntaxWarning: invalid escape sequence '\ '
  df_ventas_informacion.to_csv('csv_limpios_finales\ ventas_informacion.csv')
